In [19]:
import matplotlib.pylab as pyl
import numpy as np
import pandas as pd

In [20]:
SRC_BACKUP_EXCEL_DIR = 'E:/Diablo II Resurrected/backup/excel'
DST_EXCEL_DIR = 'E:/Diablo II Resurrected/Data/global/excel'

In [21]:
def restore_all():
    import shutil
    shutil.rmtree(DST_EXCEL_DIR)
    shutil.copytree(SRC_BACKUP_EXCEL_DIR, DST_EXCEL_DIR)

restore_all()

In [22]:
class Database:

    def __init__(self, name_of_file: str):
        self.path = f'{DST_EXCEL_DIR}/{name_of_file}.txt'
        self.data = pd.read_csv(self.path, sep="\t", dtype=object)
    
    def save(self):
        self.data.to_csv(self.path, sep='\t', mode='w', index=False)

    def get_property(self, row: int, label: str):
        return self.data.loc[row, label]

    def set_property(self, row: int, label: str, value):
        self.data.loc[row, label] = value

    def mutate_property(self, row: int, label: str, fn):
        x = self.get_property(row, label)
        self.set_property(row, label, fn(x))

In [23]:
charstats = Database('charstats')

for i in range(8):
    if i == 5: continue
    charstats.set_property(i, 'RunVelocity', 16)
    charstats.set_property(i, 'WalkVelocity', 13)
    charstats.set_property(i, 'ManaRegen', 20) # 20 seconds to reach full mana

charstats.set_property(1, 'ManaRegen', 25) # Sorc has warmth for a big boost, and generally the most mana.
charstats.set_property(0, 'ManaRegen', 15)


charstats.save()

In [24]:
hirelings = Database('hireling')

for i in range(120):
    new_hp = int(hirelings.get_property(i, 'HP')) * 10
    hirelings.set_property(i, 'HP', new_hp)

hirelings.save()

In [25]:
monstats = Database('monstats')

# 'raise skeleton' monster
monstats.set_property(363, 'Velocity', 16)
monstats.set_property(363, 'Run', 24)

monstats.set_property(363, 'Drain', 8)
monstats.set_property(363, 'Drain(N)', 15)
monstats.set_property(363, 'Drain(H)', 25)

monstats.set_property(363, 'DamageRegen', 10) # (hp * dmgRgn) / 16 = actual regen

monstats.mutate_property(363, 'minHP', lambda x: int(x) * 2)
monstats.mutate_property(363, 'maxHP', lambda x: int(x) * 2)

monstats.set_property(363, 'coldeffect', -50)
monstats.set_property(363, 'coldeffect(N)', -25)
monstats.set_property(363, 'coldeffect(H)', -10)

monstats.save()

In [26]:
skills = Database('skills')

# raise skeleton without corpse
skills.set_property(70, 'srvstfunc', '')
skills.set_property(70, 'srvdofunc', 56)
skills.set_property(70, 'cltstfunc', '')
skills.set_property(70, 'cltdofunc', '')

skills.set_property(70, 'InTown', 1)

# tp in town
skills.set_property(54, 'InTown', 1)

skills.save()

In [27]:
# frozen orb
# skills.set_property(64, 'localdelay', 10)
# skills.set_property(64, 'globaldelay', 0)

# Skill synergy; par8 = 2 aka 2%
skills.set_property(64, 'EDmgSymPerCalc', "(skill('Ice Bolt'.blvl) + skill('Cold Mastery'.blvl))*par8")

# skills.set_property(64, 'mana', 65)

skills.set_property(64, 'calc1', "ln12 * (100 + skill('Ice Bolt'.blvl) * 10 + skill('Cold Mastery'.blvl)) / 100")
skills.set_property(64, 'calc2', "ln34 * (100 + skill('Ice Bolt'.blvl) * 10 + skill('Cold Mastery'.blvl)) / 100")

skills.save()

1. Increase TP book to hold 100 scrolls.
2. Make potions heal for 10-15 seconds.
3. No stack reduction for Javelins and Arrows/Bolts.

In [28]:
# TP, ID books, and keys
misc = Database('misc')

misc.set_property(10, 'maxstack', 100)
misc.set_property(11, 'maxstack', 100)
misc.set_property(35, 'maxstack', 100)

misc.save()

In [29]:
# Arrows + Bolts
misc.set_property(18, 'minstack', 512)
misc.set_property(18, 'maxstack', 512)

misc.set_property(20, 'minstack', 512)
misc.set_property(20, 'maxstack', 512)

misc.save()

In [30]:
# Javs
weapons = Database('weapons')

weapons.set_property(47, 'minstack', 512)
weapons.set_property(47, 'maxstack', 512)

weapons.save()

In [31]:
# infinite ammo Jav
skills.set_property(2, 'srvstfunc', '')
skills.set_property(2, 'srvdofunc', '')

skills.set_property(2, 'cltstfunc', '')
skills.set_property(2, 'cltdofunc', '')

skills.set_property(2, 'srvmissile', 'javelin')
skills.set_property(2, 'cltmissile', 'javelin')

skills.set_property(2, 'noammo', 1)

# poison jav
for idx in [15, 25, 20, 35, 24, 14]:
    if skills.get_property(idx, 'srvstfunc') == '4':
        print(skills.get_property(idx, 'skill'))
        skills.set_property(idx, 'srvstfunc', '')
        skills.set_property(idx, 'srvdofunc', '')

        skills.set_property(idx, 'cltstfunc', '')
        skills.set_property(idx, 'cltdofunc', '')

    skills.set_property(idx, 'noammo', 1)
    skills.set_property(idx, 'decquant', '')


skills.save()

Poison Javelin
Plague Javelin
Lightning Bolt
Lightning Fury
